In [ ]:
SYSTEM_PROMPT = '''


'''

In [5]:
PROOF_ATTEMPT_PROMPT = ''' You are now writing a full proof of the abstract algebra problem using the chosen strategy, and are also given the problem breakdown (optional) to assist you. Your job is only to write the proof, clearly and rigorously.
This may be any iteration of trying to fix a past solution, or a different strategy, so there may optionally be critiques to be careful of.

Follow these constraints:
- Be explicit in your reasoning.
- Do not skip steps.
- Assume the reader knows abstract algebra, but write with clarity.
- avoid circular reasoning or any logical fallacies

Do not reflect on the quality of the proof. Do not critique it. Just output the best version of the proof you can.

'''

In [6]:
BREAKDOWN_PROMPT = '''
You are an abstract algebra problem solver. Your only task is to break down the problem into clearly defined parts.

Step 1: Categorize the problem — what type of problem is it? (e.g. proving irreducibility, finding ideals, checking if something is a PID, etc.)

Step 2: Preparse — what are the key objects involved (rings, elements, ideals, etc.)? What definitions or theorems might be relevant?

Step 3: Rephrase the goal in your own words, making it crystal clear what must be proven.

Only do the above three things. Do not suggest ways to solve it. Do not do any algebra.

The problem will be given straight by the user.

'''

In [7]:
IDENTIFY_AND_RANK_PROMPT = '''

You are continuing from a prior breakdown of an abstract algebra problem. Your task is to identify **distinct** ways to attempt a solution — no more than 3.

For each strategy:
- Give a 1-line name for the method.
- Give a 1–2 sentence explanation of how the method would be used to attempt the proof.

Then, rank the strategies from most promising to least promising. Use criteria such as alignment with the problem type, likelihood of success, and simplicity.

Do not perform any steps of the proof. Only identify and rank strategies.

I will send the problem first, followed by the breakdown.

Based on your reasoning, return the best proof strategy, with a brief, clear explanation of how to do it.
'''

In [8]:
CRITIQUE_PROMPT = '''
You are now evaluating a proof of an abstract algebra problem. Your task is only to critique it.
Check for:
- Logical errors
- Gaps or missing justifications
- Use of incorrect definitions
- Any unclear or imprecise statements

At the end, answer these questions:
- Is anything majorly wrong with this proof? (Yes/No)
- If yes, should we attempt a different proof strategy instead?

If after your analysis, the answer to both questions is no, explicitly say "there is nothing wrong with this".
Do not propose fixes. Just critique.


'''

In [9]:
REVISION_PROMPT = '''
You are an abstract algebra master revising a proof based on specific critiques for a given question.

Revise the original proof to directly address all valid concerns in the critique:
- Fix any logical or mathematical errors.
- Clarify unclear steps or imprecise language.
- Preserve the original strategy and structure unless the critique says it is fundamentally flawed.

Do not introduce new proof techniques unless explicitly told to. Only revise what is necessary.

'''

# How do you solve a math equation?

1. CATEGORIZE IT INTO A TYPE OF PROBLEM
2. PREPARSE THE PROBLEM - figure out what the 'objects' are, what we're broadly dealing with.
Once the category of the problem is clear, and we have an idea of the tools we are dealing with:
- break down steps to prove the question
- understand all fundamental properties of the object(s) we are dealing with
- proceed in logical steps, ensuring each step builds on the previous, with no skips or fallacies
- verify that the goal was achieved and that all assumptions were used or acknowledged

## Agent properties:
- System prompt
- messages
- breakdown of question, including understanding whats going on
- the actual solution
- the edge case detector / rigor detector
- reviewing the math and rewriting it. repeating this till we have a solution that is satisfactory.

**First node** -  breakdown the question\
**Second Node** - start thinking of some ways to do it\
**Third node** - rank the ways to solve it\
**Fourth node** - write out the proof to solve it\
**Fifth node** - critique the proof, is there anything wrong with it? rigor, clarity, communication etc.\
send it back to the previous node, till we either reach the turn limit or its saying theres no proper issues. Ask it if theres something majorly wrong, and if so, we can use another proof technique.
Return.


In [10]:
from openai import OpenAI
import os
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
client = OpenAI()



def gptHelperFunction(systemPrompt, modelName, userPrompt):
  completion = client.chat.completions.create(
    model=modelName,
    messages=[
        {"role": "system", "content": systemPrompt},
        {
            "role": "user",
            "content": userPrompt,
        },
    ],
)
  return completion.choices[0].message.content

In [14]:
class MathAgent:


  def __init__(self):
    self.breakdown = ""
    self.rankingOfSolutions = ""
    self.proofAttempts = []
    self.critiques = []

  def breakingDownProblem(self, mathQuestion):
    questionBreakdown = gptHelperFunction(BREAKDOWN_PROMPT, "gpt-4.1", mathQuestion)
    return questionBreakdown

  def identifyAndRankSolutions(self, mathQuestion, questionBreakdown):
    optimalSolutionMethod = gptHelperFunction(IDENTIFY_AND_RANK_PROMPT, "gpt-3.5-turbo",
            f"The question is: {mathQuestion}\nand the breakdown is: {questionBreakdown}")
    return optimalSolutionMethod

  def attemptingProof(self, mathQuestion, questionBreakdown, solution):
    proofAttempt = gptHelperFunction(PROOF_ATTEMPT_PROMPT, "gpt-4.1",
              f"The question is: {mathQuestion}\nand the optimal strategy decided is: {solution}"
                        )
    return proofAttempt

  def critique(self, mathQuestion, proofAttempts):
    critiques = gptHelperFunction(CRITIQUE_PROMPT, "gpt-4.1",
              f"The question is: {mathQuestion}\nand the proof is: {proofAttempts}"
                                  )
    return critiques

  def revisions(self, mathQuestion, mostRecentProofAttempt, mostRecentCritiques):
    newPrompt = """
Here is the question:
{}
Here is the original proof:
{}

Here is the critique of the proof:
{}

Revise the original proof to directly address all valid concerns
""".format(mathQuestion, mostRecentProofAttempt, mostRecentCritiques)

    revisedProof = gptHelperFunction(REVISION_PROMPT, "gpt-4.1", newPrompt)
    return revisedProof


  def __call__(self, mathQuestion):
    self.breakdown = self.breakingDownProblem(mathQuestion)
    print(self.breakdown + "\n")

    self.rankingOfSolutions = self.identifyAndRankSolutions(mathQuestion, self.breakdown)
    print(self.rankingOfSolutions + "\n")

    self.proofAttempts.append(self.attemptingProof(mathQuestion, self.breakdown, self.rankingOfSolutions))

    self.critiques.append(self.critique(mathQuestion, self.proofAttempts[-1]))

    while True:
      if "there is nothing wrong with this" in self.critiques[-1].lower():
        print(f"Criticism analysis: {self.critiques[-1]}")
        return self.proofAttempts[-1]

      self.proofAttempts.append(self.revisions(mathQuestion, self.proofAttempts[-1], self.critiques[-1]))
      self.critiques.append(self.critique(mathQuestion, self.proofAttempts[-1]))

      print(f"Most recent proof attempt: {self.proofAttempts[-1]}")
      print(f"Most recent critique: {self.critiques[-1]}")

      humanInTheLoop = input("Press quit to quit, or add some feedback: ")
      if humanInTheLoop == "quit":
        continue
      else:
        self.critiques[-1] = self.critiques[-1] + humanInTheLoop











In [15]:
myMathAgent = MathAgent()
myMathAgent("prove that a minimal polynomial is irreducible")

Step 1: Categorize the problem  
This is a proof problem, specifically about **proving irreducibility**—you are asked to prove that a certain kind of polynomial (the minimal polynomial) has the property of being irreducible.

Step 2: Preparse  
Key objects:  
- **Minimal polynomial**: Typically refers to the minimal polynomial of an element α over a field F; that is, the monic polynomial of smallest degree in \( F[x] \) such that \( f(\alpha) = 0 \).
- **Irreducible polynomial**: A non-constant polynomial in \( F[x] \) that cannot be written as a product of two non-constant polynomials in \( F[x] \).
Relevant definitions/theorems:  
- Definition of minimal polynomial.
- Definition of irreducibility in a polynomial ring over a field.
- Properties of polynomial rings, especially over fields (like \( F[x] \) being a principal ideal domain).
- Connection between roots of polynomials and divisibility.

Step 3: Rephrase the goal  
You must show that if \( m(x) \) is the minimal polynomial of

'Let \\( F \\) be a field, \\( K \\) an extension field of \\( F \\), and \\( \\alpha \\in K \\) algebraic over \\( F \\). Denote by \\( m_{\\alpha,F}(x) \\) the minimal polynomial of \\( \\alpha \\) over \\( F \\).\n\n**Claim.** The minimal polynomial \\( m_{\\alpha,F}(x) \\) is irreducible in \\( F[x] \\).\n\n**Proof:**\n\nRecall the definition of the minimal polynomial:\n\n- \\( m_{\\alpha,F}(x) \\) is monic,\n- \\( m_{\\alpha,F}(\\alpha) = 0 \\),\n- \\( m_{\\alpha,F}(x) \\) has minimal positive degree among all nonzero polynomials in \\( F[x] \\) vanishing at \\( \\alpha \\).\n\nSuppose, towards a contradiction, that \\( m_{\\alpha,F}(x) \\) is **not** irreducible in \\( F[x] \\). Then there exist nonconstant polynomials \\( f(x), g(x) \\in F[x] \\) with degrees strictly less than \\(\\deg m_{\\alpha,F}(x)\\) such that\n\\[\nm_{\\alpha,F}(x) = f(x) g(x).\n\\]\n\nSince \\( m_{\\alpha,F}(\\alpha) = 0 \\), we have\n\\[\nf(\\alpha)g(\\alpha) = 0.\n\\]\nIn a field extension, this implie